<a href="https://colab.research.google.com/github/MariiaKV/analytics/blob/main/retail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files

In [ ]:
! gdown 1HsYrBFqdUPutmwYOuiF329fP0SxIrQ_T
! gdown 1DXLBgC6oFXdA_Ar-aWLSjvdPbUV4CMn7
! gdown 1Zm8N0muSPFa-Dk6xcdRnOEC5Ug7SEhJV

Downloading...
From: https://drive.google.com/uc?id=1HsYrBFqdUPutmwYOuiF329fP0SxIrQ_T
To: /content/data1.2 pre_order.csv
100% 1.22M/1.22M [00:00<00:00, 119MB/s]
Downloading...
From: https://drive.google.com/uc?id=1DXLBgC6oFXdA_Ar-aWLSjvdPbUV4CMn7
To: /content/data1.1 YM.csv
100% 457k/457k [00:00<00:00, 75.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Zm8N0muSPFa-Dk6xcdRnOEC5Ug7SEhJV
To: /content/data1.3.fact.csv
100% 6.11M/6.11M [00:00<00:00, 95.1MB/s]


### Обработка датасета с фактическими заказами fact_bills

In [ ]:
fact_bills = pd.read_csv('/content/data1.3.fact.csv', sep=';')

In [ ]:
fact_bills.head()

,╫хъ,Name,Amount,Date
0,Q╟╩-00207626,(инд)Банкетка 160*045*046 Тк. Casanova Milk Gr...,7160,01.03.2025
1,Q╟╩-00229432,(инд)Диван,"42602,25",07.03.2025
2,Q╟╩-00224545,(инд)Изголовье Chicago (Чикаго) 210 Тк. Shammy...,"96679,002",06.03.2025
3,Q╟╩-00214752,(инд)Кровать с ПМ 120*200 Oliva Pro (Олива Про...,"8665,75",04.03.2025
4,Q╟╩-00218749,(инд)Кровать с ПМ 200*100 ORLANDO Тк. Sky Velv...,15966,04.03.2025


In [ ]:
fact_bills.dtypes

,0
╫хъ,object
Name,object
Amount,object
Date,object


Видно, что в первом столбце датасета есть артефакты, от которых нужно избавиться. Для этого, уберем все символы до первого числового значения с начала строки, чтобы остались только id. Изменим название столбцов. Приведем к соответствующему типу столбцы amount и date.

In [ ]:
fact_bills.columns = ['id','name','amount','date']

In [ ]:
# убираем дубликаты
fact_bills.drop_duplicates(inplace=True)

In [ ]:
# используем регулярное выражение для очистки строк от артефактов
fact_bills['id'] = fact_bills['id'].str.replace(r'^[^0-9]*', '', regex=True)

In [ ]:
fact_bills['date'] = pd.to_datetime(fact_bills['date'],dayfirst=True).dt.date

In [ ]:
fact_bills['amount'] = fact_bills['amount'].str.replace(',','.')
fact_bills['amount'] = fact_bills['amount'].astype('float').round(1)

In [ ]:
fact_bills.head()

,id,name,amount,date
0,00207626,(инд)Банкетка 160*045*046 Тк. Casanova Milk Gr...,7160.0,2025-03-01
1,00229432,(инд)Диван,42602.2,2025-03-07
2,00224545,(инд)Изголовье Chicago (Чикаго) 210 Тк. Shammy...,96679.0,2025-03-06
3,00214752,(инд)Кровать с ПМ 120*200 Oliva Pro (Олива Про...,8665.8,2025-03-04
4,00218749,(инд)Кровать с ПМ 200*100 ORLANDO Тк. Sky Velv...,15966.0,2025-03-04


Проверим остались ли содержащие не только числовые данные id.

In [ ]:
fact_bills[fact_bills['id'].str.contains(r'\D', na=False)]

,id,name,amount,date
14,15-00776,(н/а)Комплект стоек для матрасов Askona Disney,250.0,2025-03-05
18,15-00770,(н/а)Сумка Disney,250.0,2025-03-05
40,09347693-0282-1,EMS,785.5,2025-03-03
41,35226207-0108-1,EMS,2244.0,2025-03-05
94,15-00786,S8 Smart Light Cololight (1 pcs),262.5,2025-03-05
...,...,...,...,...
76621,"5,09219E+18",Шорты жен. Stay Cool цвет электрик размер M,429.5,2025-03-02
76634,05429660-0381-3,Шорты муж. Stay Perfect цвет серо,2245.0,2025-03-06
76635,"9,8372E+18",Шорты муж. Stay Perfect цвет серо,1145.5,2025-03-07
76666,4148475-1,Штора Blackout Basic 2.0 270*160 Антрацит,2214.0,2025-03-05


Видим, что есть значения id, которые содержат не только числа, из них представленные в виде экспаненты явно не соответствует формату id, поэтому приведем их в обычный численный вид.

In [ ]:
mask = fact_bills['id'].astype(str).str.contains('\+', na=False)
fact_bills.loc[mask, 'id'] = fact_bills.loc[mask, 'id'].str.replace(',', '.', regex=False).astype(float).astype('int64')

In [ ]:
fact_bills[fact_bills['id'].str.contains(r'\D', na=False)].head()

,id,name,amount,date
14,15-00776,(н/а)Комплект стоек для матрасов Askona Disney,250.0,2025-03-05
18,15-00770,(н/а)Сумка Disney,250.0,2025-03-05
40,09347693-0282-1,EMS,785.5,2025-03-03
41,35226207-0108-1,EMS,2244.0,2025-03-05
94,15-00786,S8 Smart Light Cololight (1 pcs),262.5,2025-03-05


### Обработка датасета c визитами пользователей ym

In [ ]:
ym = pd.read_csv('/content/data1.1 YM.csv', sep=';')

In [ ]:
ym.head()

,ClientID,дата визита на лендинг ХХХ
0,1741717280113344419,11.03.2025
1,174167984652580512,11.03.2025
2,1740417776678210403,11.03.2025
3,1741240032806135033,11.03.2025
4,1741547484141939440,11.03.2025


In [ ]:
ym.dtypes

,0
ClientID,uint64
дата визита на лендинг ХХХ,object


In [ ]:
ym.columns = ['client_id','visit_date']
ym.head()

,client_id,visit_date
0,1741717280113344419,11.03.2025
1,174167984652580512,11.03.2025
2,1740417776678210403,11.03.2025
3,1741240032806135033,11.03.2025
4,1741547484141939440,11.03.2025


In [ ]:
ym['visit_date'] = pd.to_datetime(ym['visit_date'], dayfirst=True).dt.date

### Обработка датасета c предзаказами пользователей pre_order

In [ ]:
pre_order = pd.read_csv('/content/data1.2 pre_order.csv', sep=';')

In [ ]:
pre_order.head()

,client_id,sessionid,date session_id,transactionid,transaction_date
0,1739240151359894445,512623199663423536,11.02.2025 18:11,Q鞘-00209315,01.03.2025
1,174098771993303406,958534529010630804,03.03.2025 10:41,Q资-000438305,03.03.2025
2,1732799477574480721,646689452608717061,17.02.2025 16:15,Q鞘-00228527,07.03.2025
3,1741062929927159790,1013097933830881827,05.03.2025 20:31,Q资-000441562,06.03.2025
4,1741062929927159790,1013097933830881827,05.03.2025 20:31,Q资-000441576,06.03.2025


In [ ]:
pre_order.dtypes

,0
client_id,uint64
sessionid,uint64
date session_id,object
transactionid,object
transaction_date,object


Видно, что в столбце transaction_id есть артефакты, от которых нужно избавиться. Для этого, также как и ранее уберем все символы до первого числового значения с начала строки, чтобы остались только id. Изменим название столбцов. Приведем к соответствующему типу столбцы с датами. Проверим данные на дубликаты.

In [ ]:
pre_order.rename(columns={'sessionid': 'session_id', 'date session_id':'date_session_id', 'transactionid':'transaction_id'}, inplace=True)

In [ ]:
pre_order['date_session_id'] = pd.to_datetime(pre_order['date_session_id'],dayfirst=True)
pre_order['transaction_date'] = pd.to_datetime(pre_order['transaction_date'],dayfirst=True).dt.date
pre_order['transaction_id'] = pre_order['transaction_id'].str.replace(r'^[^0-9]*','', regex=True)

In [ ]:
pre_order.head()

,client_id,session_id,date_session_id,transaction_id,transaction_date
0,1739240151359894445,512623199663423536,2025-02-11 18:11:00,00209315,2025-03-01
1,174098771993303406,958534529010630804,2025-03-03 10:41:00,000438305,2025-03-03
2,1732799477574480721,646689452608717061,2025-02-17 16:15:00,00228527,2025-03-07
3,1741062929927159790,1013097933830881827,2025-03-05 20:31:00,000441562,2025-03-06
4,1741062929927159790,1013097933830881827,2025-03-05 20:31:00,000441576,2025-03-06


In [ ]:
# находим дубликаты
pre_order.duplicated().sum()

np.int64(1703)

In [ ]:
# убираем дубликаты
pre_order.drop_duplicates(inplace=True)

In [ ]:
# проверяем transaction_id на подозрительные данные
pre_order[pre_order['transaction_id'].str.contains(r'\D', na=False)]

,client_id,session_id,date_session_id,transaction_id,transaction_date
703,1739265885481826935,507165838547878233,2025-02-11 12:24:00,00/00118,2025-03-03
1724,1695554245680069246,937649466185875508,2025-03-02 12:34:00,3/0050,2025-03-02
1781,1741147227777426670,1035475059665535316,2025-03-06 20:13:00,600/0239,2025-03-06
2422,1733725547886993335,932643754233823608,2025-03-02 07:15:00,600/0236,2025-03-04
2857,174065946747733919,889810408167702578,2025-02-28 09:51:00,1/0063,2025-03-03
4427,1739963431821730801,891440826810368073,2025-02-28 11:36:00,00/00117,2025-03-03
4451,1740036954791238563,995295996367077510,2025-03-05 01:39:00,00/00121,2025-03-06
15104,1740204971803227563,753342520412602580,2025-02-22 09:16:00,3/0052,2025-03-03


Объединяем датасеты ym и pre_order по client_id. Остаются только те посетители лендинга, которые сделали предзаказ. После чего фильтруем по дате, чтобы остались только те предзаказы клиентов, которые были сделаны после посещения ими лендинга.  

In [ ]:
ym_orders = pre_order.merge(ym, how='inner', on='client_id')

In [ ]:
ym_orders['client_id'].count()

np.int64(470)

In [ ]:
# отфильтровываем тех клиентов, которые посетили лендинг уже после того, как сделали предзаказ
ym_orders_filtered =  ym_orders[ym_orders['visit_date'] <= ym_orders['date_session_id']]

In [ ]:
ym_orders_filtered['client_id'].count()

np.int64(344)

In [ ]:
final = ym_orders_filtered.merge(fact_bills, how='inner', left_on = 'transaction_id', right_on='id')

In [ ]:
final['amount'].sum().round(2)

np.float64(2150209.2)

In [ ]:
final.head()

,client_id,session_id,date_session_id,transaction_id,transaction_date,visit_date,id,name,amount,date
0,1732786840260436100,1052636141403767086,2025-03-07 14:24:00,4160487,2025-03-07,2025-03-07,4160487,Диван,40156.4,2025-03-07
1,1732786840260436100,1052636141403767086,2025-03-07 14:24:00,4160487,2025-03-07,2025-03-07,4160487,Матрас 158*066 Carina Middle,1524.0,2025-03-07
2,1732786840260436100,1052636141403767086,2025-03-07 14:24:00,4160487,2025-03-07,2025-03-07,4160487,Матрас 160 Carina Middle,8292.5,2025-03-07
3,1732786840260436100,1052636141403767086,2025-03-07 14:24:00,4160487,2025-03-07,2025-03-07,4160487,Подушка декоративная М1 New 045*045 Тsv17 (Тк....,360.3,2025-03-07
4,1732786840260436100,1052636141403767086,2025-03-07 14:24:00,4160487,2025-03-07,2025-03-07,4160487,Подушка декоративная М1 New 070*040 Тsv17 (Тк....,1441.3,2025-03-07


### Создаем сводную таблицу для подсчета метрик

In [ ]:
final_grouped = final.groupby('client_id').agg({'transaction_id': 'nunique',
    'amount': 'sum', 'visit_date': 'min', 'date':'min'}).reset_index()

In [ ]:
final_grouped.head()

,client_id,transaction_id,amount,visit_date,date
0,17319436379267354,1,1072.5,2025-03-04,2025-03-04
1,17372271659383979,1,17254.0,2025-03-02,2025-03-02
2,17411999872591483,1,16153.0,2025-03-05,2025-03-05
3,167672469553674435,1,1185.0,2025-03-02,2025-03-02
4,171549727759696765,1,33601.7,2025-03-04,2025-03-04


В результате предобработки данных получили таблицу с данными по клиентам, которые совершили покупку после посещения лендинга, включающую в себя информацию по количеству заказов transaction_id, суммарную выручку, первую дату визита лендинга, дату оплаты первого заказа по каждому клиенту. Эту таблицу используем в качестве подключения для расчета финальной статистики и для создания на ее основе дашборда в Datalense.

In [ ]:
final_grouped.to_csv('final_grouped.csv', index=False, encoding='utf-8')

### Расчет показателей

In [ ]:
# расчет количества заказов
final_grouped['transaction_id'].sum()

np.int64(233)

In [ ]:
# расчет кол-ва участников
final_grouped['client_id'].count()

np.int64(204)

In [ ]:
# расчет выручки
final_grouped['amount'].sum()

np.float64(2150209.2)

In [ ]:
final_grouped['visit_date'] = pd.to_datetime(final_grouped['visit_date'])
final_grouped['date'] = pd.to_datetime(final_grouped['date'])

In [ ]:
final_grouped['diff_time'] = (final_grouped['date'] - final_grouped['visit_date']).dt.days

In [ ]:
# расчет среднего времени между первым посещением лендинга и покупкой в минутах
avg_time = final_grouped['diff_time'].mean()*60
avg_time

np.float64(41.470588235294116)